In [20]:
import math
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

In [21]:
cleaned_df = pd.read_csv('cleaned_shifted_data.csv')

In [22]:

# Sample the dataset
sample_size = 50000
sampled_df = cleaned_df.sample(n=sample_size, random_state=11)

# Filter relevant columns (AQI constituents)
relevant_columns = ['PM2.5 (µg/m³)', 'PM10 (µg/m³)', 'NO (µg/m³)', 'NO2 (µg/m³)',
                    'NOx (ppb)', 'NH3 (µg/m³)', 'SO2 (µg/m³)', 'CO (mg/m³)',
                    'Ozone (µg/m³)']
X = cleaned_df[relevant_columns].values
y = cleaned_df['AQI_calculated_shifted'].values
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [12]:
from dec_tree import DecisionTreeRegressor #importing the DecisionTreeRegressor class from dec_tree.py, where we have implemented it from scratch

tree = DecisionTreeRegressor()

# Fit the model
tree.fit(X_train, y_train)
# Predict the test set
y_pred = tree.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

0 0


: 

In [15]:
from sklearn.tree import DecisionTreeRegressor
# Create a decision tree classifier model object for comparison
decision_tree_regressor = DecisionTreeRegressor()

# Train the decision tree classifier model using the training data.
decision_tree_regressor.fit(X_train, y_train)

# Use the trained model to make predictions on the test data.
y_pred = decision_tree_regressor.predict(X_test)

# Evaluate the performance of the model
mse = (mean_squared_error(y_test, y_pred))
print(f"Mean Squared Error(scikit-learn): {mse}")

Mean Squared Error: 3977.107086659228


In [23]:
from sklearn.tree import DecisionTreeRegressor

class RandomForestRegressor:
    
    def __init__(self, n_estimators=10, max_depth=10, min_size=2, C=None):
        """
        Initialize the RandomForestRegressor.

        Parameters:
        - n_estimators (int): Number of decision trees in the forest.
        - max_depth (int): Maximum depth of each decision tree.
        - min_size (int): Minimum number of samples required to split a node.
        - C (int): Number of features to consider when looking for the best split.
        """
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.min_size = min_size
        self.C = C
        self.trees = []
    
    def fit(self, X, y):
        """
        Fit the RandomForestRegressor to the training data.

        Parameters:
        - X (array-like): Training input samples.
        - y (array-like): Target values.
        """
        for _ in range(self.n_estimators):
            tree = DecisionTreeRegressor(max_depth=self.max_depth)
            indices = np.random.choice(len(X), len(X), replace=True)  # Bootstrap sampling
            X_bootstrap, y_bootstrap = X[indices], y[indices]
            tree.fit(X_bootstrap, y_bootstrap)
            self.trees.append(tree)
    
    def predict(self, X_test):
        """
        Predict target values for the test data.

        Parameters:
        - X_test (array-like): Test input samples.

        Returns:
        - predictions (array-like): Predicted target values.
        """
        predictions = np.zeros((len(X_test), len(self.trees)))
        for i, tree in enumerate(self.trees):
            predictions[:, i] = tree.predict(X_test)
        return np.mean(predictions, axis=1)

In [24]:
# Initialize the random forest regressor
random_forest = RandomForestRegressor(n_estimators=100, max_depth=10, min_size=2, C=None)

# Fit the random forest model to the training data
random_forest.fit(X_train, y_train)

# Make predictions on the test data
y_pred = random_forest.predict(X_test)

# Evaluate the performance of the model (e.g., using mean squared error)
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

KeyboardInterrupt: 

In [17]:
from sklearn.ensemble import RandomForestRegressor

# Initialize the random forest regressor
sklearn_random_forest = RandomForestRegressor(n_estimators=100, max_depth=10, min_samples_split=2)

# Fit the random forest model to the training data
sklearn_random_forest.fit(X_train, y_train)

# Make predictions on the test data
y_pred_sklearn = sklearn_random_forest.predict(X_test)

# Evaluate the performance of the scikit-learn random forest model (e.g., using mean squared error)
mse_sklearn = mean_squared_error(y_test, y_pred_sklearn)
print("Mean Squared Error (scikit-learn RandomForestRegressor):", mse_sklearn)

Mean Squared Error (scikit-learn RandomForestRegressor): 3942.6876404304044
